<a href="https://colab.research.google.com/github/adowaconan/Deep_learning_fMRI/blob/master/5_3_demonstrate_the_DNN_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Just installing some of the necessary libraries to get the data and process the data

In [0]:
!pip install -U -q PyDrive
!pip install tqdm
!apt-get install swig
!pip install -U pymvpa2
!pip install -U seaborn

# this lets you access to googel drive shared files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 13 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 3s (329 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_3.0.12-1_amd64.deb ...
Unpacking 

# Copying the word list and the word2vec model from Google Drive to Colab, so it will take some time.

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


word_list_id = 'https://drive.google.com/open?id=18nfVy-o0GWX-QKEWrKK0EKLLAltpFy4U'.split('id=')[-1]
word2vec_model_id = 'https://drive.google.com/open?id=1Dj9aTqHbuknWZC9kQJabqH3NqGXe_PT4'.split('id=')[-1]

word_list = drive.CreateFile({'id':word_list_id})
word2vec_model_ = drive.CreateFile({'id':word2vec_model_id})

word_list.GetContentFile('{}'.format('word.npy'))
word2vec_model_.GetContentFile('{}'.format('glove-sbwc.i25.vec'))

In [0]:
roi_id = 'https://drive.google.com/open?id=10cAoPizl69QR2RekIKpkdrv9J4_Emcq-'.split('id=')[-1]
roi = drive.CreateFile({'id':roi_id})
roi.GetContentFile('{}'.format('roi.pkl'))

# The python libraries we will need to import

1. numpy: numerical python
2. pandas: table
3. tqdm: progress bar
4. **gensim**: need for loading word2vec models
5. scipy.spatial.distance: representational dissimilarity matrix
6. MinMaxScaler: rescale features to between 0 and 1
7. LinearSVC: a selected linear classifier
8. roc_auc_score: a selected scoring method, good for balanced or unbalanced data
9. CalibratedClassifierCV: a classifier wrapper for providing probabilistic predictions
10. MultiOutputClassifier: a classifier wrapper for decoding if target is more than 1-D
11. model_selection: CV method
12. seaborn (better in 0.9.0 version): pretty plotting
13. pickle: to load the dataset (PYMVPA format) that is stored in a pickle dump file

In [0]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from gensim.models.keyedvectors import KeyedVectors # for loading word2vec models
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import StratifiedShuffleSplit,LeavePGroupsOut
import seaborn as sns
sns.set_style('white')
sns.set_context('poster')
import pickle

In [0]:
# candidate model
Glove_from_SBWC = 'glove-sbwc.i25.vec'
# load the stimuli table
words = np.load('word.npy')
print('loading Glove model, and it is going to take some time...')
model_glove = KeyedVectors.load_word2vec_format(Glove_from_SBWC)

loading Glove model, and it is going to take some time...


# I lost track of how many keras functions I imported, so bear with me.

In [0]:
import keras
from keras.layers                                  import Dense,Dropout,BatchNormalization
from keras.layers                                  import Input
from keras.layers                                  import Flatten,LeakyReLU,merge
from keras.layers                                  import GaussianNoise,GaussianDropout

from keras.models                                  import Model,Sequential
from keras.layers.noise                            import AlphaDropout
from keras.callbacks                               import ModelCheckpoint,TensorBoard,EarlyStopping
from keras                                         import backend as K
from keras                                         import regularizers
from keras.layers                                  import Reshape
from keras.layers                                  import Conv3DTranspose,Activation, Lambda,Multiply
from keras.layers                                  import Layer
from keras.losses                                  import mse

from mvpa2.datasets.base                           import Dataset
from mvpa2.mappers.fx                              import mean_group_sample

from sklearn.metrics                               import roc_auc_score,roc_curve
from sklearn.metrics                               import (classification_report,
                                                           matthews_corrcoef,
                                                           confusion_matrix,
                                                           f1_score,
                                                           log_loss)
from sklearn.preprocessing                         import MinMaxScaler

from sklearn.model_selection                       import StratifiedShuffleSplit,LeaveOneGroupOut
from sklearn.preprocessing                         import OneHotEncoder

Using TensorFlow backend.


# Tons of helper functions, so that I don't have to repeat myself

In [0]:
def get_blocks(dataset__,label_map,key_type='labels'):
    """
    # use ids, chunks,and labels to make unique blocks of the pre-average dataset, because I don't want to 
    # average the dataset until I actually want to, but at the same time, I want to balance the data for 
    # both the training and test set.
    """
    ids                     = dataset__.sa.id.astype(int)
    chunks                  = dataset__.sa.chunks
    words                   = dataset__.sa.words
    if key_type == 'labels':
        try: # in metasema
            labels              = np.array([label_map[item] for item in dataset__.sa.targets])[:,-1]
        except:# not in metasema
            labels              = np.array([label_map[item] for item in dataset__.sa.targets])
        
    elif key_type == 'words':
        labels              = np.array([label_map[item] for item in dataset__.sa.words])
    sample_indecies         = np.arange(len(labels))
    blocks                  = [np.array([ids[ids             == target],
                                         chunks[ids          == target],
                                         words[ids           == target],
                                         labels[ids          == target],
                                         sample_indecies[ids == target]
                                         ]) for target in np.unique(ids)]
    block_labels            = np.array([np.unique(ll[-2]) for ll in blocks]).ravel()
    return blocks,block_labels

def customized_partition(dataset__,label_map):
    """
    Similar to the function above, but it nests with cross validation process,
    making sure each word instance will appear in the test set for at least once
    """
    unique_words = np.unique(dataset__.sa.words)
    unique_chunks = np.unique(dataset__.sa.chunks)
    try: # in metasema
        labels              = np.array([label_map[item] for item in dataset__.sa.targets])[:,-1]
    except:# not in metasema
        labels              = np.array([label_map[item] for item in dataset__.sa.targets])
    words = dataset__.sa.words
    chunks = dataset__.sa.chunks
    blocks,block_labels = get_blocks(dataset__,label_map,key_type='labels')
    sample_indecies         = np.arange(len(labels))
    test = []
    check = []
    for n in range(int(1e5)):
        random_chunk = np.random.choice(unique_chunks,size=1,replace=False)[0]
        working_words = words[chunks == random_chunk]
        working_block = [block for block in blocks if (int(np.unique(block[1])[0]) == random_chunk)]
        random_word = np.random.choice(working_words,size=1,replace=False)[0]
        if random_word not in check:
            for block in working_block:
                if (np.unique(block[2])[0] == random_word) and (random_word not in check):
                    test.append(block[-1].astype(int))
                    check.append(block[2][0])
#                    print(test,check)
                if len(check) == len(unique_words):
                    break
            if len(check) == len(unique_words):
                break
        if len(check) == len(unique_words):
            break
    test = np.concatenate(test,0).flatten()
    train = np.array([idx for idx in sample_indecies if (idx not in test)])
    return train,test

class stacked_perceptron(object):
    """
    to construct stacked layers of dense-batchnormaization-activation-layers
    edit: batch normalization is no used due to activation SELU
    https://towardsdatascience.com/selu-make-fnns-great-again-snn-8d61526802a9
    https://arxiv.org/pdf/1706.02515.pdf
    """
    def __init__(self,
#                 inputs,
                 hidden_units, # number of neurons of the dense layer
                 layer_func                                 = Dense,
                 beta                                       = 1, # L1 regularization value
                 lamda                                      = 1,# L2 regularization, used exclusively in parameters regularization
                 dropout_func                               = AlphaDropout,# special drop out function: https://arxiv.org/pdf/1706.02515.pdf
                 dropout_rate                               = 0.2,# dropout rate
                 name                                       = [1,1,'encode'],# the way of naming the layers
                 kernel_regularizer                         = regularizers.l1_l2,# parameter regularization function: elatic
                 kernel_initializer                         ='he_normal',# initialize the parameter method
                 activation                                 = 'selu',# self normalizing NN
                 alpha                                      = 0.3, # not used
                 l1                                         = False, # default of not using L1 activation regularization
                 use_bias                                   = True,# defult of using bias terms in the dense layers
                 batch_normal                               = True,# defult of using batch normalization
                 ):
#        self.inputs = inputs
        self.layer_func                                     = layer_func
        self.hidden_units                                   = hidden_units
        self.kernel_regularizer                             = kernel_regularizer
        self.kernel_initializer                             = kernel_initializer
        self.beta                                           = beta
        self.lamda                                          = lamda
        self.dropout_func                                   = dropout_func
        self.dropout_rate                                   = dropout_rate
        self.name                                           = name
        self.activation                                     = activation
        self.alpha                                          = alpha
        self.l1                                             = l1
        self.use_bias                                       = use_bias
        self.batch_normal                                   = batch_normal
    def __call__(self,net):
        ####################### dense layer block ###################################################
        if self.l1: # if regularize the activation output
            net = self.layer_func(units                     = self.hidden_units,
                        use_bias                            = self.use_bias,
                        kernel_initializer                  = self.kernel_initializer,
                        kernel_regularizer                  = self.kernel_regularizer(1e-6,self.lamda),
                        activity_regularizer                = regularizers.l1(self.beta),
                        name                                = 'dense_layer{}_{}_{}'.format(self.name[0],
                                                                                           self.name[1],
                                                                                           self.name[2])
                        )(net)
        else:
            net = self.layer_func(units                     = self.hidden_units,
                        use_bias                            = True,
                        kernel_initializer                  = self.kernel_initializer,
                        kernel_regularizer                  = self.kernel_regularizer(1e-6,self.lamda),
#                        activity_regularizer = regularizers.l1(self.beta),
                        name                                = 'dense_layer{}_{}_{}'.format(self.name[0],
                                                                                           self.name[1],
                                                                                           self.name[2])
                        )(net)
        #################### end of dense layer block #################################################
        #################### batch normalization block ################################################
        if self.batch_normal:
            net = BatchNormalization(name                   = 'norm_layer{}_{}_{}'.format(self.name[0],
                                                                                          self.name[1]+1,
                                                                                          self.name[2]))(net)
        ################### end of batch normalization block ################################################
        ################### actiavtion block ##########################################################
        if type(self.activation) is not str:
            net = self.activation(name                      = 'activation_layer{}_{}_{}'.format(self.name[0],
                                                                                                self.name[1]+2, 
                                                                                                self.name[2]),
                                  alpha                     = self.alpha)(net)
        else:
            net = Activation(activation = self.activation,
                             name                           = 'activation_layer{}_{}_{}'.format(self.name[0],
                                                                                                self.name[1]+2, 
                                                                                                self.name[2]),
                              )(net)
        ################### end of activation block ######################################################
        ################### dropout layer block ##########################################################
        net = self.dropout_func(rate                        = self.dropout_rate,
                      name                                  = 'drop_layer{}_{}_{}'.format(self.name[0],
                                                                                          self.name[1]+3,
                                                                                          self.name[2]))(net)
        
        return net
def make_CallBackList(model_name,monitor='val_loss',mode='min',verbose=0,min_delta=1e-4,patience=50,frequency = 1):
    """
    Make call back function lists for the keras models
    
    Inputs
    -------------------------
    model_name: directory of where we want to save the model and its name
    monitor: the criterion we used for saving or stopping the model
    mode: min --> lower the better, max --> higher the better
    verboser: printout the monitoring messages
    min_delta: minimum change for early stopping
    patience: temporal windows of the minimum change monitoring
    frequency: temporal window steps of the minimum change monitoring
    
    Return
    --------------------------
    CheckPoint: saving the best model
    EarlyStopping: early stoppi....
    """
    checkPoint = ModelCheckpoint(model_name,# saving path
                                 monitor          =monitor,# saving criterion
                                 save_best_only   =True,# save only the best model
                                 mode             =mode,# saving criterion
                                 period           =frequency,# frequency of check the update 
                                 verbose          =verbose# print out (>1) or not (0)
                                 )
    earlyStop = EarlyStopping(   monitor          =monitor,
                                 min_delta        =min_delta,
                                 patience         =patience,
                                 verbose          =verbose, 
                                 mode             =mode,
                                 )
    return [checkPoint,earlyStop]
def clf(shape,
         beta               = 1,
         lamda              = 1,
         dropout_rate       = 0.,
         latent_dim         = 36,
         lr                 = 1e-3,
         decay              = 0,
         add_noise          = False,
         add_sparsity       = False,
         rho                = 0.02,
         sparse_beta        = 1,
         output_shape       = 2,
         
         ):
    inputs                  = Input(shape                                   = (shape,),
                              batch_shape                                   = (None,shape),
                              name                                          = 'inputs',
                              dtype                                         = 'float32'
                              )
    ####################################### add noise to the inputs #################################
    ############################ Gaussian noise #####################################################
    if add_noise:
        inputs_            = GaussianNoise(stddev = 1)(inputs)
        encode_            = inputs_
    else:
        encode_            = inputs
    if type(latent_dim) is int:
        latent_dims = [latent_dim]
    else:
        latent_dims = latent_dim
    for jj,latent_dim_ in enumerate(latent_dims):
        np.random.seed(12345)
        encode_           = stacked_perceptron(latent_dim_,
                                              beta                          = beta,
                                              lamda                         = lamda,
                                              dropout_rate                  = dropout_rate,
                                              name                          = [jj+1,jj+1,'latent'],
                                              use_bias                      = True,
                                              batch_normal                  = True,
                                              activation                    = 'sigmoid',
                                              )(encode_)
        if add_sparsity:
            encode_       = _sparse_reg(rho = rho, beta = sparse_beta)(encode_)
    decode_                = encode_
    np.random.seed(12345)
    outputs                = Dense(units                                    = output_shape,
                             use_bias                                       = True,
                             name                                           ='outputs',
                             kernel_initializer                             = 'he_normal',
                             kernel_regularizer                             = regularizers.l1_l2(beta,lamda),
                             activity_regularizer                           = regularizers.l1(1),
                             activation                                     = 'softmax',# constraint between 0 and 1
                             )(decode_)
    model                  = Model(inputs, outputs, name = 'classifier')
    model.compile(optimizer= keras.optimizers.Adam(lr = lr,decay = decay),
                  loss     = keras.losses.categorical_crossentropy,
                  metrics  = [keras.metrics.categorical_accuracy]
                  )
    word_outputs = Dense(units                                          = latent_dim,
                         use_bias                                       = True,
                         name                                           ='outputs',
                         kernel_initializer                             = 'he_normal',
                         kernel_regularizer                             = regularizers.l1_l2(beta,lamda),
                         activity_regularizer                           = regularizers.l1(1),
                         activation                                     = 'softmax',# constraint between 0 and 1
                         )(decode_)
    word_model = Model(inputs,word_outputs,name = 'word_classifier')
    word_model.compile(optimizer = keras.optimizers.Adam(lr = lr / 10., decay = decay * 1000.),
                       loss      = keras.losses.categorical_crossentropy,
                       metrics   = [keras.metrics.categorical_accuracy]
                       )
    return model,word_model
def DNN_train_validate(dataset_train,
                       dataset_test,
                       label_map,
                       output_dir,
                       sub_name,
                       roi_name,
                       fold,
                       patience,
                       batch_size,
                       epochs,
                       print_train,
                       classifier,
                       flip = False,
                       model_type = 'DNN',
                       n_splits = 5,
                ):
    tr = dataset_train
    X_train = tr.samples.astype('float32')
    groups = tr.sa.chunks
    te = dataset_test
    te = te.get_mapped(mean_group_sample(['chunks', 'id'],order = 'occurrence'))
    X_test = te.samples.astype('float32')
    print('train on {} and test on {}'.format(tr.shape[0],te.shape[0]))
    y_train = np.array([label_map[item] for item in tr.targets])
    y_test = np.array([label_map[item] for item in te.targets])
    if flip:
        X_train_flip = np.fliplr(X_train)
        X_train = np.concatenate([X_train,X_train_flip])
        y_train = np.concatenate([y_train,y_train])
        groups = np.concatenate([groups,groups])
    
    
    # define the scaler
    scaler          = MinMaxScaler() # scale the data between 0 and 1
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    onehot = OneHotEncoder()
    if y_train.shape[-1] == 2:
        labels_train = y_train
        labels_test = y_test
    else:
        labels_train = onehot.fit_transform(y_train.reshape(-1,1)).toarray()
        labels_test = onehot.transform(y_test.reshape(-1,1)).toarray()
    
    
    # define the model name and saving directory
    dnn_model_name   = 'temp.hdf5' # temporal model name 
    # we will need 2 call back functions: 1) in-train saving and 2) early stopping
    callBackList = make_CallBackList(dnn_model_name,
                                           monitor                      = 'val_{}'.format(classifier.metrics_names[-1]), # metric I care
                                           mode                         = 'max', # how I care
                                           verbose                      = 0,# print out the process
                                           min_delta                    = 1e-5,# how much difference it should make
                                           patience                     = patience, # early stop argument
                                           frequency                    = 1
                                           )
    # first, feed the autoencoder 0s, and see what the raw-raw gradient values are
    # a trick I learned from twitter
    print('initial loss {:.4f}'.format(classifier.evaluate(np.zeros(X_train.shape),
                                                        np.zeros((X_train.shape[0],labels_train.shape[1])),
                                                        batch_size  = batch_size,
                                                        verbose     = 0
                                                        )[0]
                                    )
                    )
    
    blocks_tr,block_labels    = get_blocks(dataset_train,label_map)
    cv = StratifiedShuffleSplit(n_splits=n_splits,test_size=.2,random_state=12345)
    iterator = cv.split(blocks_tr,block_labels)
    for z,(training,validation) in enumerate(iterator):
        idx_training    = np.concatenate([blocks_tr[tr_el][-1] for tr_el in training]).astype(int)
        idx_validation  = np.concatenate([blocks_tr[te_el][-1] for te_el in validation]).astype(int)
        print('{} loss starts with {:.4f} on source'.format(model_type,
                            classifier.evaluate(
                                              X_train,
                                              labels_train,
                                              batch_size                    = batch_size,
                                              verbose                       = 0
                                                                      )[0]))
        from sklearn import utils as sk_utils
        X,y = sk_utils.shuffle(X_train[idx_training],labels_train[idx_training])
        history                 = classifier.fit(  
                                        X,y,# input, output
                                        batch_size                              = batch_size,# min batch size
                                        epochs                                  = epochs,# just lots of epochs
                                        validation_data                         = (X_train[idx_validation],labels_train[idx_validation]),# split the last 10% of the data as the validation data
                                        verbose                                 = print_train,# print out the process
                                        callbacks                               = callBackList # call back functions
                                        )

        classifier.load_weights(dnn_model_name) # load the saved best model weights
        # let's look at the current model loss with all the training + validation data combined
        training_loss = classifier.evaluate(X_train,labels_train,
                                             batch_size                     = batch_size,
                                             verbose                        = 0
                                             )
        print('{} current loss = {:.4f},{} = {:.4f} on source'.format(
              model_type,
              training_loss[0],
              classifier.metrics_names[-1],
              training_loss[-1]))
    
    classifier.load_weights(dnn_model_name)
    return (classifier,
            scaler,
            onehot,
            dnn_model_name,
            history,
#            (score_baseline1,score_baseline2,score_baseline3,score_baseline4),
#            (tn,fp,fn,tp),
            X_train,
            labels_train,
            X_test,
            labels_test,
            )

In [0]:
label_map               = dict(animal  =[0,1],
                               tool    =[1,0])
sub                     = '*'       # star means all subjects
average                 = False      # averaging the trainig data
transfer                = False     # do I do domain adaptation
concatenate             = False     # specifically for domain adaptation
flip                    = True,     # double the size of the training data
n_splits                = 10        # number of cross validation
test_size               = 0.2       # proprotion of the test data
patience                = 70 # if after this many epochs, the loss or the accuracy does not change, we stop the training
epochs                  = 3000 # just use a large number to prompt early stop
print_train             = 0 # 1 - print the progress bar, 0 - don't
batch_size              = 64
beta                    = 0.
lamda                   = 0.
learning_rate           = 1e-4
decay                   = 1e-9
dropout_rate            = 0.5
latent_dim              = 36

In [0]:
ds = pickle.load(open('roi.pkl','rb'))

In [0]:
condition = 'read'

In [0]:
dataset = ds[ds.sa.context == condition]

In [0]:
blocks, block_labels    = get_blocks(dataset,label_map)
labels                  = np.array([label_map[item] for item in dataset.targets])
cv                      = StratifiedShuffleSplit(n_splits       = n_splits,
                                                 test_size      = test_size,
                                                 random_state   = 12345)

In [0]:
np.random.seed(12345)
chunks = dataset.sa.chunks
used_test = []
fold = -1
a = []
output_dir = ''
sub_name = 'test'
roi_name = 'roi'
for _ in range(1000):
    print('paritioning ...')
    idx_train,idx_test = customized_partition(dataset,label_map,)
    if any([np.sum(idx_test) == np.sum(item) for item in used_test]):
        pass
    else:
        fold += 1
        used_test.append(idx_test)
        print('done')
        
        
        # must do!!! Otherwise, it will run out of resource eventualy
        K.clear_session()
        shape       = dataset.shape[1]
        # control the initialization of the DNN model
        np.random.seed(12345)
        classifier,_= clf(
                                  shape, # input shape
                                  dropout_rate          = dropout_rate, # drop out rate for each hidden layer
                                  beta                  = beta, # for sparsity
                                  lamda                 = lamda, # for model simplicity
                                  lr                    = learning_rate, # learning rate
                                  decay                 = decay, # schedule decay of the learning rate 
                                  latent_dim            = latent_dim, # latent space dimensionality 
                                  )

        (classifier,
        scaler,
        onehot,
        dnn_model_name,
        history,
        X_train,
        labels_train,
        X_test,
        labels_test,
        )                        = DNN_train_validate(
                                        dataset_train   = dataset[idx_train],
                                        dataset_test    = dataset[idx_test],
                                        classifier      = classifier,
                                        label_map       = label_map,
                                        output_dir      = output_dir,
                                        sub_name        = sub_name,
                                        roi_name        = roi_name,
                                        fold            = fold,
                                        patience        = patience,
                                        batch_size      = batch_size,
                                        epochs          = epochs,
                                        print_train     = print_train,
                                        flip            = True,
                                        n_splits        = 5,
                                        )
        pred_                   = classifier.predict(X_test)
        score_baseline1         = np.array([roc_auc_score(labels_test[:,ii],pred_[:,ii]) for ii in range(labels_test.shape[-1])])
        a.append(score_baseline1.mean())

paritioning ...
done
 * Please note: warnings are printed only once, but underlying problem might occur many times *
train on 1323 and test on 36
initial loss 63.6531
DNN loss starts with 64.5410 on source
DNN current loss = 64.3109,categorical_accuracy = 0.6531 on source
DNN loss starts with 64.3109 on source
DNN current loss = 64.2384,categorical_accuracy = 0.6803 on source
DNN loss starts with 64.2384 on source
DNN current loss = 64.2384,categorical_accuracy = 0.6803 on source
DNN loss starts with 64.2384 on source
DNN current loss = 64.2346,categorical_accuracy = 0.6852 on source
DNN loss starts with 64.2346 on source
DNN current loss = 64.2291,categorical_accuracy = 0.6882 on source
paritioning ...
done
train on 1323 and test on 36
initial loss 63.6531
DNN loss starts with 64.5455 on source
DNN current loss = 64.2501,categorical_accuracy = 0.7173 on source
DNN loss starts with 64.2501 on source
DNN current loss = 64.1943,categorical_accuracy = 0.7249 on source
DNN loss starts with

KeyboardInterrupt: ignored

In [0]:
np.mean(a)

0.5166666666666667